# Register abstraction

The CoHDL standard library defines an abstraction layer that allows us to describe MMIO register devices as hierarchical structures of Python classes.

These structures can be connected to interfaces such as AXI (Wishbone or Avalon should also be possible but are not implemented yet) or exported to a [SystemRDL](https://github.com/SystemRDL) file for documentation.


In [1]:
# This codeblock defines an AxiBaseEntity. It connect AXI-lite signals
# to a register map defined in the abstract method gen_addr_map.
# 
# The examples below inherit from this Entity and overload gen_addr_map.
#
# The introduced library abstracts interface specific operations.
# This means, that you could implement a WishboneBaseEntity or an AvalonBaseEntity
# and reuse all examples unchanged.

from __future__ import annotations

import cohdl
from cohdl import Port, Bit, BitVector, Signal, Unsigned, Null, Full
from cohdl import std
from cohdl.std.axi import axi4_light as axi
from cohdl.std.reg import reg32

from typing import Annotated as Ann
from typing import Self

class AxiBaseEntity(cohdl.Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    axi_awaddr = Port.input(Unsigned[32])
    axi_awprot = Port.input(Unsigned[3])
    axi_awvalid = Port.input(Bit)
    axi_awready = Port.output(Bit, default=Null)

    axi_wdata = Port.input(BitVector[32])
    axi_wstrb = Port.input(BitVector[4])
    axi_wvalid = Port.input(Bit)
    axi_wready = Port.output(Bit, default=Null)

    axi_bresp = Port.output(BitVector[2], default=Null)
    axi_bvalid = Port.output(Bit, default=Null)
    axi_bready = Port.input(Bit)

    axi_araddr = Port.input(Unsigned[32])
    axi_arprot = Port.input(Unsigned[3])
    axi_arvalid = Port.input(Bit)
    axi_arready = Port.output(Bit, default=Null)

    axi_rdata = Port.output(BitVector[32], default=Null)
    axi_rresp = Port.output(BitVector[2], default=Null)
    axi_rvalid = Port.output(Bit, default=Null)
    axi_rready = Port.input(Bit)

    def architecture(self):
        clk = std.Clock(self.clk)
        reset = std.Reset(self.reset)

        axi_con = axi.Axi4Light(
            clk=clk,
            reset=reset,
            wraddr=axi.Axi4Light.WrAddr(
                valid=self.axi_awvalid,
                ready=self.axi_awready,
                awaddr=self.axi_awaddr,
                awprot=self.axi_awprot,
            ),
            wrdata=axi.Axi4Light.WrData(
                valid=self.axi_wvalid,
                ready=self.axi_wready,
                wdata=self.axi_wdata,
                wstrb=self.axi_wstrb,
            ),
            wrresp=axi.Axi4Light.WrResp(
                valid=self.axi_bvalid,
                ready=self.axi_bready,
                bresp=self.axi_bresp,
            ),
            rdaddr=axi.Axi4Light.RdAddr(
                valid=self.axi_arvalid,
                ready=self.axi_arready,
                araddr=self.axi_araddr,
                arprot=self.axi_arprot,
            ),
            rddata=axi.Axi4Light.RdData(
                valid=self.axi_rvalid,
                ready=self.axi_rready,
                rdata=self.axi_rdata,
                rresp=self.axi_rresp,
            ),
        )

        # connect the AXI signals to the address map
        # defined in the exampled derived below
        axi_con.connect_addr_map(self.gen_addr_map())
    
    def gen_addr_map(self) -> reg32.AddrMap:
        # Inherit from AxiBaseEntity and override this method
        # to define an address map to connect to the AXI interface.
        raise NotImplementedError()

## AddrMap and Memory

Every register map consists of one class derived from `reg32.AddrMap`. The register layout is defined using type annotations.

In the following example an `ExampleAddrMap`, containing two memory regions is defined. The magic method `_config_` is called each time the class is instantiated. It is used to configure additional parameters of the memories.

In [2]:
from __future__ import annotations

from cohdl.std.reg import reg32

class ExampleAddrMap(reg32.AddrMap):
    # Type annotations of AddrMap define the address layout.
    # ExampleAddrMap contains two memory blocks.

    ro_memory: reg32.RoMemory[0x0000:0x1000]    # read-only block of 0x1000 bytes at address 0x0000
    wr_memory: reg32.Memory[0x2000:0x2800]      # writable block of 0x800 bytes at address 0x2000

    def _config_(self):
        # An optional _config_ method is used to pass additional parameters
        # to register objects.
        
        # Populate read only memory with incrementing unsigned values.
        # _config_ is executed as normal Python code
        # so configurations could also be loaded from the file system.
        self.ro_memory._config_(initial=[Unsigned[32](nr) for nr in range(0x1000//4)])

        # Define initial state of writable memory as all ones.
        # Set `noreset` to True so the memory keeps its content
        # when the axi interface is reset.
        self.wr_memory._config_(initial=Full, noreset=True)

class ExampleEntity(AxiBaseEntity):
    # ExampleEntity is derived from the previously defined AxiBaseEntity.
    # Overloads `gen_addr_map` connect the register implementation to AXI.
    def gen_addr_map(self):
        return ExampleAddrMap()

print(std.VhdlCompiler.to_string(ExampleEntity))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity ExampleEntity is
  port (
    clk : in std_logic;
    reset : in std_logic;
    axi_awaddr : in unsigned(31 downto 0);
    axi_awprot : in unsigned(2 downto 0);
    axi_awvalid : in std_logic;
    axi_awready : out std_logic;
    axi_wdata : in std_logic_vector(31 downto 0);
    axi_wstrb : in std_logic_vector(3 downto 0);
    axi_wvalid : in std_logic;
    axi_wready : out std_logic;
    axi_bresp : out std_logic_vector(1 downto 0);
    axi_bvalid : out std_logic;
    axi_bready : in std_logic;
    axi_araddr : in unsigned(31 downto 0);
    axi_arprot : in unsigned(2 downto 0);
    axi_arvalid : in std_logic;
    axi_arready : out std_logic;
    axi_rdata : out std_logic_vector(31 downto 0);
    axi_rresp : out std_logic_vector(1 downto 0);
    axi_rvalid : out std_logic;
    axi_rready : in std_logic
    );
end ExampleEntity;


architecture arch_ExampleEntity of ExampleEntity is
  function cohdl_bool_to_std

## Registers with user defined logic

In the previous example we defined two memory regions using the predefined types `reg32.Memory` and `reg32.RoMemory`. It is also possible to create custom register types with user defined behavior. The most convenient way to do this, is to inherit from `reg32.Register`. `reg32.Register` represents a single register word of 32 Bits. Similar to the overall layout of the address map, the registers structure is defined using type annotations (of `reg32.*Field`).


In [3]:
from __future__ import annotations

from cohdl.std.reg import reg32

class RegOr(reg32.Register):
    # Type annotations of Register define the layout of the register.

    # MemFields keep their value after each write operation.
    inp_a: reg32.MemField[7:0]
    inp_b: reg32.MemField[15:8]

    # Normal fields are not automatically assigned new values.
    # User logic like the following _impl_concurrent_ function
    # can set their value.
    output: reg32.Field[31:24]

    # When _impl_concurrent_ is defined it is evaluated in a
    # std.concurrent context. This example performs an or operation
    # of the two input fields an assigns the result to the output.
    def _impl_concurrent_(self) -> None:
        self.output <<= self.inp_a.val() | self.inp_b.val()

class AddrMap(reg32.AddrMap):
    # this AddrMap contains three instances of RegOr
    # at the memory offsets 0x00, 0x10 and 0x14

    reg_0: RegOr[0x00]
    reg_1: RegOr[0x10]
    reg_2: RegOr[0x14]

class ExampleEntity(AxiBaseEntity):
    def gen_addr_map(self):
        return AddrMap()

print(std.VhdlCompiler.to_string(ExampleEntity))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity ExampleEntity is
  port (
    clk : in std_logic;
    reset : in std_logic;
    axi_awaddr : in unsigned(31 downto 0);
    axi_awprot : in unsigned(2 downto 0);
    axi_awvalid : in std_logic;
    axi_awready : out std_logic;
    axi_wdata : in std_logic_vector(31 downto 0);
    axi_wstrb : in std_logic_vector(3 downto 0);
    axi_wvalid : in std_logic;
    axi_wready : out std_logic;
    axi_bresp : out std_logic_vector(1 downto 0);
    axi_bvalid : out std_logic;
    axi_bready : in std_logic;
    axi_araddr : in unsigned(31 downto 0);
    axi_arprot : in unsigned(2 downto 0);
    axi_arvalid : in std_logic;
    axi_arready : out std_logic;
    axi_rdata : out std_logic_vector(31 downto 0);
    axi_rresp : out std_logic_vector(1 downto 0);
    axi_rvalid : out std_logic;
    axi_rready : in std_logic
    );
end ExampleEntity;


architecture arch_ExampleEntity of ExampleEntity is
  function cohdl_bool_to_std

## SystemRDL

It is possible, to export design documentation in the SystemRDL format. The document contains

* the layout of the register device
* Python doc strings
* meta data added using typing.Annotated

The last example in this notebook contains a more complex example and shows, how to further process SystemRDL to HTML.

In [4]:
from cohdl.std.reg import to_system_rdl

class RegBinOp(reg32.Register):
    """
    The doc strings of register objects are added to the SystemRDL documentation.

    This register is a generalization of the previous RegOr. The binary operation
    it performs is defined at compile time in _config_.
    """

    inp_a: reg32.MemField[7:0]
    inp_b: reg32.MemField[15:8]

    # The declaration of register objects can be wrapped in a typing.Annotated object.
    # This has no effect on the generated VHDL but the metadata (in this case the
    # read-only memory attribute and a info string)
    # will show up in the generated SystemRDL documentation.
    output: Ann[reg32.Field[31:24], reg32.Access.r, "this text describes the output field"]

    def _config_(self, operation):
        # the optional config method is used to define the type of
        # operation performed by this register
        self.op = operation

    def _impl_concurrent_(self) -> None:
        self.output <<= self.op(self.inp_a.val(), self.inp_b.val())

class SystemRdlExample(reg32.AddrMap):
    """
    This address map contains multiple binary registers
    and is documented in Python.
    """

    # this AddrMap contains three instances of RegOr
    # at the memory offsets 0x00, 0x10 and 0x14

    reg_or: RegBinOp[0x00]
    reg_xor: RegBinOp[0x10]
    reg_and: RegBinOp[0x14]

    def _config_(self):
        self.reg_or._config_(lambda a, b: a|b)
        self.reg_xor._config_(lambda a, b: a^b)
        self.reg_and._config_(lambda a, b: a&b)

# generate SystemRDL documentation
print(to_system_rdl(SystemRdlExample()))

addrmap SystemRdlExample {
  name = "SystemRdlExample";
  desc = "This address map contains multiple binary registers
    and is documented in Python.";
  default regwidth = 32;
  
  
  reg {
    desc = "The doc strings of register objects are added to the SystemRDL documentation.

    This register is a generalization of the previous RegOr. The binary operation
    it performs is defined at compile time in _config_.";
    
    field {
    } inp_a [7:0];
    
    field {
    } inp_b [15:8];
    
    field {
      desc = "this text describes the output field";
      sw = r;
    } output [31:24];
  } reg_or @ 0x0;
  
  
  reg {
    desc = "The doc strings of register objects are added to the SystemRDL documentation.

    This register is a generalization of the previous RegOr. The binary operation
    it performs is defined at compile time in _config_.";
    
    field {
    } inp_a [7:0];
    
    field {
    } inp_b [15:8];
    
    field {
      desc = "this text describes the output 

## RegFiles

`RegFiles` are collections of register objects, they contain `RegisterObjects` (like `reg32.Register`) or nested `RegFiles`. `AddrMap` is just a special case of a `RegFile` with the ability to connect to bus interfaces.

In [5]:
class ClkDevice(reg32.RegFile, word_count=2):
    """
    Defines a counter, that is incremented synchronous to the
    bus clock. The counter value can be used as an accurate time source.
    """

    class Ctrl(reg32.Register):
        """
        contains the control parameters of the clock device
        """

        enable: Ann[reg32.MemField[0, Null], "set this field to '1' to start the clock"]
        prescaler: Ann[reg32.MemUField[31:16, 1], "prescaler factor"]

    ctrl: Ctrl[0]
    count: reg32.UWord[4]
    
    def _config_(self):
        self._counter = Signal[Unsigned[32]](0)
    
    def _impl_sequential_(self) -> None:
        # _impl_sequential_ is similar to _impl_concurrent_
        # it is evaluated in its own sequential context and inherits
        # clock/reset from the bus interface

        if self.ctrl.enable:
            if self._counter >= self.ctrl.prescaler.val():
                self.count <<= self.count.raw + 1
                self._counter <<= 0
            else:
                self._counter <<= self._counter + 1

In [6]:
class RgbPwm(reg32.Register):
    """
    controls the brightness of the three color channels of an rgb LED
    """

    r: Ann[reg32.MemUField[7:0, 0], "controls strength of color red"]
    g: Ann[reg32.MemUField[15:8, 0], "controls strength of color green"]
    b: Ann[reg32.MemUField[23:16, 0], "controls strength of color blue"]
    
    def _config_(self, out_rgb: Signal[BitVector[3]]):
        self._out = out_rgb
        self._full_cnt = Signal[Unsigned[16]](0)
        self._cnt = self._full_cnt.msb(8).unsigned
    
    def _impl_sequential_(self):
        self._full_cnt <<= self._full_cnt + 1
        self._out[0] <<= self._cnt < self.r.val()
        self._out[1] <<= self._cnt < self.g.val()
        self._out[2] <<= self._cnt < self.b.val()

In [7]:
class SlowTask(reg32.Register):
    """
    Example for a register that performs a slow task.
    Write '1' to 'flag' to start the task. This bit will be cleared
    automatically once the task is done and the register is ready
    to process more data.
    """

    input: Ann[reg32.MemField[15:0], "input data of the operations"]
    flag: Ann[reg32.FlagField[16], "write '1' to start operation"]
    
    async def _impl_sequential_(self):
        # wait util '1' is written to bit 16 of this register
        # `flag` will read as '1' until it is explicitly cleared
        await self.flag.is_set()

        # perform some long running task
        counter = Signal[Unsigned[16]](self.input.val())
        while counter:
            counter <<= counter - 1
        
        # clear the flag bit to indicate, that the register
        # is not longer busy
        self.flag.clear()
    
    async def _impl_sequential_(self):
        # equivalent to previous function but using
        # async with to wait-for and clear the flag

        async with self.flag:
            counter = Signal[Unsigned[16]](self.input.val())

            while counter:
                counter <<= counter - 1

In [8]:
class Notifications(reg32.Register):
    """
    This register demonstrates, how notification types can be used to react to
    register reads/writes. The fields `read_count` and `write_count` are incremented
    every time the respective operation is performed on the register.
    `data` is a normal register field that can be read and written.
    """

    data: reg32.MemField[15:0, Null]
    read_count: Ann[reg32.UField[23:16, 0], reg32.Access.r, "incremented each time the register is read"]
    write_count: Ann[reg32.UField[31:24, 0], reg32.Access.r, "incremented each time the register is written"]

    notify_read: reg32.PushOnNotify.Read
    notify_write: reg32.PushOnNotify.Write

    def _impl_sequential_(self):
        if self.notify_read:
            # self.rd_notification is true for one clock cycle after each
            # read from the current register
            self.read_count <<= self.read_count.val() + 1
        if self.notify_write:
            # self.wr_notification is true for one clock cycle after each
            # write to the current register
            self.write_count <<= self.write_count.val() + 1


## customizing read and write behavior

It is also possible to define the register behavior inline by overriding the methods `_on_read_` and `_on_write_` (with `def` or `async def` functions).
These methods are called in the context of the corresponding memory interface transaction. If they are defined as coroutines stretching multiple clock ticks, the response will be delayed.

### `_on_write_`

* is called for each write access to the register
* receives the incoming data parsed into an instance of Self as its only argument
* returns an instance of Self, this value is used to update the MemFields/FlagFields of the register. For registers without such fields the return value is ignored an can be set to None.
* the default version of this function returns the incoming data unchanged

### `_on_read_`

* is called for each read access to the register
* returns an instance of Self, this value is serialized and sent back in the read response
* the default version of this function returns `self`

In [9]:
class CustomizedRegister(reg32.Register):
    """
    This register demonstrates an alternative way to implement read/write behavior
    of register types. It overloads the methods _on_read_ and _on_write_
    to inject code into the bus transaction itself.

    _on_write_ is called for every write access to the current register address.
    The received data is converted to in instance of Self and used as the single argument.
    The returned value is used to update MemFields and FlagFields.

    _on_read_ is called for every read access to the current register address.
    It takes no arguments. The returned value - an instance of Self - is converted
    to a 32-Bit vector and passed to the read response.
    """

    field_a: reg32.MemField[7:0]
    field_b: reg32.MemField[15:8]

    output: reg32.Field[31:24]

    def _config_(self):
        self._wr_count = Signal[Unsigned[8]](Null, name="wr_count")
        self._rd_count = Signal[Unsigned[8]](Null, name="rd_count")

    async def _on_write_(self: Self, data: Self) -> Self | None:
        self._wr_count <<= self._wr_count + 1

        # The call operator of registers returns a copy of the register.
        # All fields specified in the argument list are replaced with
        # the given value.

        # Here we are returning a copy of data with the value of field_a
        # inverted for demonstration purposes
        return data(
            field_a=~data.field_a.val()
        )

    async def _on_read_(self: Self) -> Self:
        self._rd_count <<= self._rd_count + 1
        
        # The read operation returns the access counters in field_a
        # and field_b.
        # Output is the result of an or operation of the previously inverted
        # field_a and field_b.
        return self(
            field_a=self._wr_count,
            field_b=self._rd_count,
            output=self.field_a.val()|self.field_b.val()
        )

In [10]:
from cohdl.std.reg import reg32

class AddrMap(reg32.AddrMap):
    """
    This example address map contains all register types defined in this notebook.

    Use std.VhdlCompiler to turn it into synthesizable VHDL.
    Use to_systemd_rdl to generate html documentation or C headers.
    """

    reg_or: RegOr[0x00]
    
    reg_bin_and: Ann[RegBinOp[0x10], "this is the and operation"]
    reg_bin_or: Ann[RegBinOp[0x14], "this is the or operation"]
    reg_bin_xor: Ann[RegBinOp[0x18], "this is the xor operation"]

    clk: ClkDevice[0x20]
    rgb: RgbPwm[0x30]
    slow: SlowTask[0x40]
    notify: Notifications[0x50]
    customized: CustomizedRegister[0x60]

    ro_memory: reg32.RoMemory[0x1000:0x1100]
    rw_memory: reg32.Memory[0x2000:0x2800]

    def _config_(self, rgb: Signal[BitVector[3]]):
        self.reg_bin_and._config_(lambda a,b: a&b)
        self.reg_bin_or._config_(lambda a,b: a|b)
        self.reg_bin_xor._config_(lambda a,b: a^b)

        self.rgb._config_(rgb)

        self.ro_memory._config_([
            Unsigned[32](val) for val in range(0x100 // 4)
        ])



class FullEntity(AxiBaseEntity):
    # define an additional port for the PWM modulated RGB output
    rgb = Port.output(BitVector[3])

    def gen_addr_map(self):
        return AddrMap(self.rgb)

# generate SystemRDL documentation and write it to a file
with open("output.rdl", "w") as file:
    print(to_system_rdl(AddrMap), file=file)

# use peakrdl to generate HTML documentation from SystemRDL
!peakrdl html output.rdl -o html_output

# turn design into VHDL
print(std.VhdlCompiler.to_string(FullEntity))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity FullEntity is
  port (
    clk : in std_logic;
    reset : in std_logic;
    axi_awaddr : in unsigned(31 downto 0);
    axi_awprot : in unsigned(2 downto 0);
    axi_awvalid : in std_logic;
    axi_awready : out std_logic;
    axi_wdata : in std_logic_vector(31 downto 0);
    axi_wstrb : in std_logic_vector(3 downto 0);
    axi_wvalid : in std_logic;
    axi_wready : out std_logic;
    axi_bresp : out std_logic_vector(1 downto 0);
    axi_bvalid : out std_logic;
    axi_bready : in std_logic;
    axi_araddr : in unsigned(31 downto 0);
    axi_arprot : in unsigned(2 downto 0);
    axi_arvalid : in std_logic;
    axi_arready : out std_logic;
    axi_rdata : out std_logic_vector(31 downto 0);
    axi_rresp : out std_logic_vector(1 downto 0);
    axi_rvalid : out std_logic;
    axi_rready : in std_logic;
    rgb : out std_logic_vector(2 downto 0)
    );
end FullEntity;


architecture arch_FullEntity of FullEntity